In [24]:
import pandas as pd
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
import numpy as np
import time
from sklearn import metrics
import pickle
import numpy as np
import requests
import os
import pandas as pd
import csv

from io import BytesIO
from matplotlib.pyplot import imshow
from PIL import Image

import rasterio
import rasterio.features
import rasterio.warp
from rasterio import MemoryFile
from PIL import ImageColor
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm

In [15]:
!pip install rasterio


     |████████████████████████████████| 18.1MB 174kB/s 


In [3]:
df1 = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Updated_CSV_2.csv")
df2 = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Updated_CSV_Random.csv")

In [5]:
df2.head()

,Unnamed: 0,Latitude,Longitude,ID,Time,Locust,Soil_Moisture,Wind,Vegetation_Index,Humidity_D,Humidity_N,Elevation,Temperature_D,Temperature_N
0,0,53.228092,-105.125951,22258.0,2016-10-05,0.0,#7dbe00,#f4f4f4,#f4f4f4,#f4f4f4,#f4f4f4,#41bf36,#f4f4f4,#f4f4f4
1,1,30.680808,-98.742585,22259.0,2016-04-14,0.0,#00ab57,#393d26,#393d26,#393d26,#393d26,#28b93f,#393d26,#393d26
2,2,46.139354,-90.550267,22260.0,2016-09-06,0.0,#00c489,#9a9f98,#9a9f98,#9a9f98,#9a9f98,#44c035,#9a9f98,#9a9f98
3,3,37.734402,-72.824027,22261.0,2016-06-25,0.0,#77787c,#77787c,#77787c,#77787c,#77787c,#77787c,#77787c,#77787c
4,4,25.817881,-110.647170,22262.0,2016-06-25,0.0,#7f7f7d,#7f7f7d,#7f7f7d,#7f7f7d,#7f7f7d,#7f7f7d,#7f7f7d,#7f7f7d


In [6]:
df1.head()

,Row,Latitude,Longitude,ID,Time,Locust,Soil_Moisture,Wind,Vegetation_Index,Humidity_D,Humidity_N,Elevation,Temperature_D,Temperature_N
0,0.0,-12.959167,19.792500,22258.0,10/5/2016,1.0,#ffe300,#636967,#636967,#636967,#636967,#cfc40a,#636967,#636967
1,1.0,49.514444,16.706667,22259.0,4/14/2016,1.0,#00b164,#e7e7e7,#e7e7e7,#e7e7e7,#e7e7e7,#4ac233,#e7e7e7,#e7e7e7
2,2.0,43.128889,10.635278,22260.0,9/6/2016,1.0,#ffec00,#f5dcdf,#f5dcdf,#f5dcdf,#f5dcdf,#57c52e,#f5dcdf,#f5dcdf
3,3.0,44.483333,16.316667,22261.0,6/25/2016,1.0,#00bd7c,#cccecb,#cccecb,#cccecb,#cccecb,#cbca0a,#cccecb,#cccecb
4,4.0,44.250000,15.016667,22262.0,6/25/2016,1.0,#3b9d00,#29333d,#29333d,#29333d,#29333d,#29333d,#29333d,#29333d


In [7]:
df = (df1.loc[:370]).append(df2.loc[:295])
df

,Row,Latitude,Longitude,ID,Time,Locust,Soil_Moisture,Wind,Vegetation_Index,Humidity_D,Humidity_N,Elevation,Temperature_D,Temperature_N,Unnamed: 0
0,0.0,-12.959167,19.792500,22258.0,10/5/2016,1.0,#ffe300,#636967,#636967,#636967,#636967,#cfc40a,#636967,#636967,NaN
1,1.0,49.514444,16.706667,22259.0,4/14/2016,1.0,#00b164,#e7e7e7,#e7e7e7,#e7e7e7,#e7e7e7,#4ac233,#e7e7e7,#e7e7e7,NaN
2,2.0,43.128889,10.635278,22260.0,9/6/2016,1.0,#ffec00,#f5dcdf,#f5dcdf,#f5dcdf,#f5dcdf,#57c52e,#f5dcdf,#f5dcdf,NaN
3,3.0,44.483333,16.316667,22261.0,6/25/2016,1.0,#00bd7c,#cccecb,#cccecb,#cccecb,#cccecb,#cbca0a,#cccecb,#cccecb,NaN
4,4.0,44.250000,15.016667,22262.0,6/25/2016,1.0,#3b9d00,#29333d,#29333d,#29333d,#29333d,#29333d,#29333d,#29333d,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
291,NaN,38.007016,-68.952521,22549.0,2019-07-02,0.0,#1b222c,#1b222c,#1b222c,#1b222c,#1b222c,#1b222c,#1b222c,#1b222c,291.0
292,NaN,46.041518,-112.812577,22550.0,2019-07-02,0.0,#fff800,#eaebed,#eaebed,#eaebed,#eaebed,#d1bf0a,#eaebed,#eaebed,292.0
293,NaN,37.917815,-80.370140,22551.0,2019-07-02,0.0,#0094ff,#677576,#677576,#677576,#677576,#a8d913,#677576,#677576,293.0
294,NaN,25.506995,-101.439732,22552.0,2019-07-02,0.0,#ffe300,#f6f6f6,#f6f6f6,#f6f6f6,#f6f6f6,#cacc0b,#f6f6f6,#f6f6f6,294.0


In [9]:
df.shape

(667, 15)

In [10]:
dataX = df[df.columns[[6,11]]]
dataY = df[df.columns[[5]]]

In [11]:
dataX.head()

,Soil_Moisture,Elevation
0,#ffe300,#cfc40a
1,#00b164,#4ac233
2,#ffec00,#57c52e
3,#00bd7c,#cbca0a
4,#3b9d00,#29333d


Converting the hex code to rgb components and storing them

In [12]:
count = 0
r,g,b,r1, g1, b1 = [],[],[],[],[],[]
for index, row in dataX.iterrows():
  print(count)
  hex = dataX.iloc[count][0]
  rgb =  ImageColor.getcolor(str(hex), "RGB")
  r.append(rgb[0])
  g.append(rgb[1])
  b.append(rgb[2])
  hex = dataX.iloc[count][1]
  rgb =  ImageColor.getcolor(str(hex), "RGB")
  r1.append(rgb[0])
  g1.append(rgb[1])
  b1.append(rgb[2])
  count += 1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [13]:
len(r1)

667

In [14]:
new_columns = ["R_soil", "G_soil", "B_soil", "R_ele", "G_ele", "B_ele"]

In [15]:
df[new_columns[0]] = r
df[new_columns[1]] = g
df[new_columns[2]] = b
df[new_columns[3]] = r1
df[new_columns[4]] = g1
df[new_columns[5]] = b1

In [16]:
df.head()

,Row,Latitude,Longitude,ID,Time,Locust,Soil_Moisture,Wind,Vegetation_Index,Humidity_D,Humidity_N,Elevation,Temperature_D,Temperature_N,Unnamed: 0,R_soil,G_soil,B_soil,R_ele,G_ele,B_ele
0,0.0,-12.959167,19.792500,22258.0,10/5/2016,1.0,#ffe300,#636967,#636967,#636967,#636967,#cfc40a,#636967,#636967,NaN,255,227,0,207,196,10
1,1.0,49.514444,16.706667,22259.0,4/14/2016,1.0,#00b164,#e7e7e7,#e7e7e7,#e7e7e7,#e7e7e7,#4ac233,#e7e7e7,#e7e7e7,NaN,0,177,100,74,194,51
2,2.0,43.128889,10.635278,22260.0,9/6/2016,1.0,#ffec00,#f5dcdf,#f5dcdf,#f5dcdf,#f5dcdf,#57c52e,#f5dcdf,#f5dcdf,NaN,255,236,0,87,197,46
3,3.0,44.483333,16.316667,22261.0,6/25/2016,1.0,#00bd7c,#cccecb,#cccecb,#cccecb,#cccecb,#cbca0a,#cccecb,#cccecb,NaN,0,189,124,203,202,10
4,4.0,44.250000,15.016667,22262.0,6/25/2016,1.0,#3b9d00,#29333d,#29333d,#29333d,#29333d,#29333d,#29333d,#29333d,NaN,59,157,0,41,51,61


In [17]:
X = df[df.columns[[15,16,17,18,19,20]]]


In [18]:
dataY.astype(int)

,Locust
0,1
1,1
2,1
3,1
4,1
...,...
291,0
292,0
293,0
294,0


Train-Test Split

In [60]:
X_train, X_test, y_train, y_test = train_test_split(X, dataY, test_size=0.2)

Random Forest Classifier

In [61]:
clf=RandomForestClassifier(n_estimators=100)

In [62]:
X_train

,R_soil,G_soil,B_soil,R_ele,G_ele,B_ele
188,97,101,110,97,101,110
290,105,108,115,105,108,115
266,255,229,0,208,170,12
175,59,157,0,29,39,49
244,255,241,0,22,34,46
...,...,...,...,...,...,...
270,182,176,178,182,176,178
259,0,180,106,0,134,50
326,62,71,76,62,71,76
150,130,193,0,0,162,69


In [63]:
y_train = list(y_train[y_train.columns[0]])
y_test = list(y_test[y_test.columns[0]])

In [64]:
clf = clf.fit(X_train,y_train)

In [65]:
y_pred = clf.predict(X_test)

In [66]:
print("Training Accuracy:",metrics.accuracy_score(y_train, clf.predict(X_train)))
print("Test Accuracy:",metrics.accuracy_score(y_test, y_pred))
base_accuracy = metrics.accuracy_score(y_test, y_pred)

Training Accuracy: 0.9924953095684803
Test Accuracy: 0.6716417910447762


In [67]:
cm = metrics.confusion_matrix(y_test,y_pred)

In [68]:
cm

array([[37, 17],
       [27, 53]])

Hyperparameter Tuning

In [70]:
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 2000, num = 10)]
max_features = ['auto', 'sqrt']
max_depth = [int(x) for x in np.linspace(5, 110, num = 10)]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4, 10]
bootstrap = [True, False]
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)

{'n_estimators': [100, 311, 522, 733, 944, 1155, 1366, 1577, 1788, 2000], 'max_features': ['auto', 'sqrt'], 'max_depth': [5, 16, 28, 40, 51, 63, 75, 86, 98, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4, 10], 'bootstrap': [True, False]}


In [73]:
start_time = time.time()
rf = RandomForestClassifier()
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
rf_random.fit(X_train, y_train)
print("Total time taken--- %s seconds---" % (time.time() - start_time))

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:   49.5s
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed:  3.1min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  5.7min finished


Total time taken--- 345.847177028656 seconds---


In [74]:
def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    accuracy = metrics.accuracy_score(test_labels, predictions)
    print('Model Performance')
    print('Accuracy = {:0.2f}%.'.format(accuracy))
    
    return accuracy, predictions

In [75]:
best_random = rf_random.best_estimator_
random_accuracy, preds = evaluate(best_random, X_test, y_test)
random_accuracy

Model Performance
Accuracy = 0.69%.


0.6865671641791045

In [76]:
print('Improvement of {:0.2f}%.'.format( 100 * (random_accuracy - base_accuracy) / base_accuracy))

Improvement of 2.22%.


In [77]:
cm = metrics.confusion_matrix(y_test,preds)
cm

array([[24, 30],
       [12, 68]])

In [80]:
pickle.dump(best_random, open("best_rf_model.sav", 'wb'))

**Support** **Vector** **Machines**

In [94]:
svc = svm.SVC(kernel='rbf') 

svc.fit(X_train, y_train)

y_pred = svc.predict(X_test)
accuracy = metrics.accuracy_score(y_test, y_pred)
print(accuracy)

0.6865671641791045


In [95]:
print("Precision:",metrics.precision_score(y_test, y_pred))
print("Recall:",metrics.recall_score(y_test, y_pred))

Precision: 0.7021276595744681
Recall: 0.825


In [98]:
print("Train accuracy", metrics.accuracy_score(y_train, svc.predict(X_train)))
print("Test accuracy", accuracy)
print(metrics.confusion_matrix(y_test,y_pred))
print(metrics.classification_report(y_test,y_pred))

Train accuracy 0.6791744840525328
Test accuracy 0.6865671641791045
[[26 28]
 [14 66]]
              precision    recall  f1-score   support

         0.0       0.65      0.48      0.55        54
         1.0       0.70      0.82      0.76        80

    accuracy                           0.69       134
   macro avg       0.68      0.65      0.66       134
weighted avg       0.68      0.69      0.68       134



Hyperparameter tuning

In [32]:
param_grid = {'C': [0.1,1, 10, 100], 'gamma': [1,0.1,0.01,0.001],'kernel': ['rbf', 'poly', 'sigmoid']}

In [ ]:
start_time = time.time()
svm = svm.SVC()
grid = GridSearchCV(svm,param_grid,verbose=2, cv = 3)
grid.fit(X_train,y_train)
print("Total time taken--- %s seconds---" % (time.time() - start_time))

Fitting 3 folds for each of 48 candidates, totalling 144 fits
[CV] C=0.1, gamma=1, kernel=rbf ......................................
[CV] ....................... C=0.1, gamma=1, kernel=rbf, total=   0.0s
[CV] C=0.1, gamma=1, kernel=rbf ......................................
[CV] ....................... C=0.1, gamma=1, kernel=rbf, total=   0.0s
[CV] C=0.1, gamma=1, kernel=rbf ......................................
[CV] ....................... C=0.1, gamma=1, kernel=rbf, total=   0.0s
[CV] C=0.1, gamma=1, kernel=poly .....................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


In [ ]:
print(grid.best_estimator_)
best_svm = grid.best_estimator_

In [68]:
#pickle.dump(clf, open("rf_model.sav", 'wb'))

In [99]:
pickle.dump(svc, open("BEST_SVM_model.sav", 'wb'))


In [4]:
feature_list = ['Soil_Moisture', 'Elevation', 'Soil_Temperature', 'Vegetation_Index']

**Python API function to use this model**

In [5]:
List_URL = [
"https://wvs.earthdata.nasa.gov/api/v1/snapshot?REQUEST=GetSnapshot&TIME={}&BBOX={}&CRS=EPSG:4326&LAYERS=VIIRS_SNPP_CorrectedReflectance_TrueColor,SMAP_L4_Analyzed_Surface_Soil_Moisture&WRAP=day,day&FORMAT=image/tiff&WIDTH=380&HEIGHT=385&ts=1601748025134",
"https://wvs.earthdata.nasa.gov/api/v1/snapshot?REQUEST=GetSnapshot&TIME={}&BBOX={}&CRS=EPSG:4326&LAYERS=VIIRS_SNPP_CorrectedReflectance_TrueColor,ASTER_GDEM_Color_Index&WRAP=day,day&FORMAT=image/tiff&WIDTH=380&HEIGHT=385&ts=1601748025134",
"https://wvs.earthdata.nasa.gov/api/v1/snapshot?REQUEST=GetSnapshot&TIME={}&BBOX={}&CRS=EPSG:4326&LAYERS=VIIRS_SNPP_CorrectedReflectance_TrueColor,SMAP_L4_Soil_Temperature_Layer_1&WRAP=day,day&FORMAT=image/tiff&WIDTH=380&HEIGHT=385&ts=1601748025134",
"https://wvs.earthdata.nasa.gov/api/v1/snapshot?REQUEST=GetSnapshot&TIME={}&BBOX={}&CRS=EPSG:4326&LAYERS=VIIRS_SNPP_CorrectedReflectance_TrueColor,MODIS_Terra_NDVI_8Day&WRAP=day,day&FORMAT=image/tiff&WIDTH=380&HEIGHT=385&ts=1601748025134",
]

#1:SMAP_L4_Analyzed_Surface_Soil_Moisture
#2:ASTER_GDEM_Color_Index (Elevation)
#3:SMAP_L4_Soil_Temperature_Layer_1 (Soil Temp)
#4 MODIS_Terra_NDVI_8Day

In [30]:
def get_val(lat, longo, time):
  unit_length = 0.06
  lower_lat = lat - unit_length
  left_long = longo - unit_length

  upper_lat = lat + unit_length
  right_long = longo + unit_length 

  Bbox = (lower_lat,left_long,upper_lat,right_long)

  Bbox = str(Bbox)[1:]

  Bbox = Bbox[:-1]

  Bbox = Bbox.replace(" ", "")
  df = pd.DataFrame(columns = feature_list)
  for i in range(len(List_URL)):
      
      link = List_URL[i]
      url = link.format(time, Bbox)
      response = requests.get(url)
      print(url)
      img = BytesIO(response.content)

      with open("example2.tif", "wb") as  f:
        f.write(img.getvalue())

      with rasterio.open('example2.tif') as dataset:
        mask = dataset.dataset_mask()
        
        for geom, val in rasterio.features.shapes(
                mask, transform=dataset.transform):
  
            geom = rasterio.warp.transform_geom(
              dataset.crs, 'EPSG:4326', geom, precision=6)
     
            b = dataset.index(longo, lat)
            a = dataset.read()
            index = a[:,b[0]-1,b[1]]
            index = '#%02x%02x%02x' % (index[0], index[1],index[2])
            
            df.loc[0, feature_list[i]] = index
  return df

In [3]:
new_columns = ["R_soil", "G_soil", "B_soil", "R_ele", "G_ele", "B_ele","R_temp", "G_temp", "B_temp", "R_veg", "G_veg", "B_veg",]

In [28]:
def getRBGValues(df):
  count = 0
  r,g,b,r1, g1, b1,r2, g2, b2,r3, g3, b3 = [],[],[],[],[],[],[],[],[],[],[],[]
  for index, row in df.iterrows():
    print(count)
    hex = df.iloc[count][0]
    print("1",df)
    rgb =  ImageColor.getcolor(str(hex), "RGB")
    r.append(rgb[0])
    g.append(rgb[1])
    b.append(rgb[2])
    print("2",hex)
    hex = df.iloc[count][1]
    rgb =  ImageColor.getcolor(str(hex), "RGB")
    r1.append(rgb[0])
    g1.append(rgb[1])
    b1.append(rgb[2])
    hex = df.iloc[count][2]
    rgb =  ImageColor.getcolor(str(hex), "RGB")
    r2.append(rgb[0])
    g2.append(rgb[1])
    b2.append(rgb[2])
    hex = df.iloc[count][3]
    rgb =  ImageColor.getcolor(str(hex), "RGB")
    r3.append(rgb[0])
    g3.append(rgb[1])
    b3.append(rgb[2])
    count += 1
    df[new_columns[0]] = r
  df[new_columns[1]] = g
  df[new_columns[2]] = b
  df[new_columns[3]] = r1
  df[new_columns[4]] = g1
  df[new_columns[5]] = b1
  df[new_columns[6]] = r2
  df[new_columns[7]] = g2
  df[new_columns[8]] = b2
  df[new_columns[9]] = r3
  df[new_columns[10]] = g3
  df[new_columns[11]] = b3
  return df

In [32]:
def load_model():
  model = pickle.load("model.sav")
  return model

Just call this function with latitude, longitude and date to check for locust invasion

In [19]:
def get_locust_prob(lat, longo):
  df = get_val(lat, longo, "2019-01-01")
  df = getRBGValues(df)
  #model = load_model()  
  prediction = model.predict(df)
  print(prediction)

API test

In [31]:
lat = 9.57
longo = 45.46
get_locust_prob(lat, longo)

https://wvs.earthdata.nasa.gov/api/v1/snapshot?REQUEST=GetSnapshot&TIME=2019-01-01&BBOX=9.51,45.4,9.63,45.52&CRS=EPSG:4326&LAYERS=VIIRS_SNPP_CorrectedReflectance_TrueColor,SMAP_L4_Analyzed_Surface_Soil_Moisture&WRAP=day,day&FORMAT=image/tiff&WIDTH=380&HEIGHT=385&ts=1601748025134
https://wvs.earthdata.nasa.gov/api/v1/snapshot?REQUEST=GetSnapshot&TIME=2019-01-01&BBOX=9.51,45.4,9.63,45.52&CRS=EPSG:4326&LAYERS=VIIRS_SNPP_CorrectedReflectance_TrueColor,ASTER_GDEM_Color_Index&WRAP=day,day&FORMAT=image/tiff&WIDTH=380&HEIGHT=385&ts=1601748025134
https://wvs.earthdata.nasa.gov/api/v1/snapshot?REQUEST=GetSnapshot&TIME=2019-01-01&BBOX=9.51,45.4,9.63,45.52&CRS=EPSG:4326&LAYERS=VIIRS_SNPP_CorrectedReflectance_TrueColor,SMAP_L4_Soil_Temperature_Layer_1&WRAP=day,day&FORMAT=image/tiff&WIDTH=380&HEIGHT=385&ts=1601748025134
https://wvs.earthdata.nasa.gov/api/v1/snapshot?REQUEST=GetSnapshot&TIME=2019-01-01&BBOX=9.51,45.4,9.63,45.52&CRS=EPSG:4326&LAYERS=VIIRS_SNPP_CorrectedReflectance_TrueColor,MODIS_Terr